# The crypto challenge
https://cryptopals.com/

第五部分開始!!

# Set5

## Diffie Hellman
### Challenge 33 - Implementing

將 "p" 設為 37, "g" 設為 5

產生一個mod 37的random number "a". 接下來計算 ``A = (g**a) % p``

同理計算 "b" 和 "B".

"A" 和 "B" 為 public keys. 計算 ``s = (B**a) % p`` 來產生session key

接下來計算 ``A**b``, 理論上這兩者相等~

你可以把s hash成 128 bit當成key

或用 SHA256 產生 encrypting key 和 MAC key.

現在NIST的參數試試看吧~~
```
p:
ffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024
e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd
3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec
6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f
24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361
c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552
bb9ed529077096966d670c354e4abc9804f1746c08ca237327fff
fffffffffffff
 
g: 2
```


In [1]:
from Crypto.Random import random

def doDiffieHellman(p, g):
    a = random.randint(0, p)
    A = pow(g, a, p)
    b = random.randint(0, p)
    B = pow(g, b, p)
    return (a, A, b, B)

p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
g = 2
a, A, b, B = doDiffieHellman(p, g)
print(a, A, b, B)
sA = pow(B, a, p)
sB = pow(A, b, p)
if sA != sB:
    print(str(sA) + ' != ' + str(sB))
print(sA)

(1813093508227616774240591551232606131609895628462451008161471459335002188083468389702060528093605528881803334378148449734104112841244433648370943311193917363974582923886209272352907679572203946197608890209669251231364282082607815281315853454682330265679249943503804876496048020001439827620195261028853741773332823037740660210155571064556457099283362802872920194912162039766000283989036698561040092606963123919247754986567732319313062798035948415798320517572731291L, 381547492637679696438879962426456508321467817483429031723744989362722535515988810550343883546748022327068814031771250430489966557321666835541229106590379611886853515672108512180661610503837466620475895371449346928607353055528511487240767329613647172890467026601481042602264080639739603708406221920264290382234153429940046601458712999631851524511971146576521319853509629938489020993413046182634064959841756305084471019995028492202243639891623175002014127654151590L, 15470497459095353143435399413445247302626571816370850481583697472916

### challenge34 - Implement a MITM key-fixing attack on Diffie-Hellman with parameter injection
### challenge35 - Implement DH with negotiated groups, and break with malicious "g" parameters 

是man in the middle，要同時撰寫client, server和 attacker的script有興趣可以挑戰!!

### Challenge 36 - Implement Secure Remote Password (SRP)

類似用DH產生 AES key。

以  C and S (client & server) 取代 A 和 B

C & S
同意 N=[NIST Prime], g=2, k=3, I (email), P (password)

S
產生隨機 salt 
計算 xH=SHA256(salt|password)
轉換 xH 成 x somehow (put 0x on hexdigest)
計算 v=g**x % N

除 x, xH，存下其他東西

C->S
Send I, A=g**a % N (a la Diffie Hellman)

S->C
Send salt, B=kv + g**b % N

S, C
Compute string uH = SHA256(A|B), u = integer of uH
C
Generate string xH=SHA256(salt|password)
Convert xH to integer x somehow (put 0x on hexdigest)
Generate S = (B - k * g**x)**(a + u * x) % N
Generate K = SHA256(S)
S
Generate S = (A * v**u) ** b % N
Generate K = SHA256(S)
C->S
Send HMAC-SHA256(K, salt)
S->C
Send "OK" if HMAC-SHA256(K, salt) validates
You're going to want to do this at a REPL of some sort; it may take a couple tries.

It doesn't matter how you go from integer to string or string to integer (where things are going in or out of SHA256) as long as you do it consistently. I tested by using the ASCII decimal representation of integers as input to SHA256, and by converting the hexdigest to an integer when processing its output.

This is basically Diffie Hellman with a tweak of mixing the password into the public keys. The server also takes an extra step to avoid storing an easily crackable password-equivalent.

### Challenge 37 - Break SRP with a zero key 

Get your SRP working in an actual client-server setting. "Log in" with a valid password using the protocol.

Now log in without your password by having the client send 0 as its "A" value. What does this to the "S" value that both sides compute?

Now log in without your password by having the client send N, N*2, &c.

Cryptanalytic MVP award

### Challenge 38 - Offline dictionary attack on simplified SRP

## RSA

### Challenge 39 - Implement RSA

Finally, to encrypt a string, do something cheesy, like convert the string to hex and put "0x" on the front of it to turn it into a number. The math cares not how stupidly you feed it strings.

In [3]:
from Crypto.Random import random
import binascii

def to_bytes(n, length, endianess='big'):
    h = '%x' % n
    s = ('0'*(len(h) % 2) + h).zfill(length*2).decode('hex')
    return s if endianess == 'big' else s[::-1]

def randbytes(k):
    return to_bytes(random.getrandbits(8*k), k)


def invmod(a, n):
    t = 0
    newt = 1
    r = n
    newr = a
    while newr != 0:
        q = r // newr
        (t, newt) = (newt, t - q * newt)
        (r, newr) = (newr, r - q * newr)
    if r > 1:
        raise Exception('unexpected')
    if t < 0:
        t += n
    return t

smallPrimes = [2, 3, 5, 7, 11, 13, 17, 19]

def hasSmallPrimeFactor(p):
    for x in smallPrimes:
        if p % x == 0:
            return True
    return False

def isProbablePrime(p, n):
    for i in range(n):
        a = random.randint(1, p)
        if pow(a, p - 1, p) != 1:
            return False
    return True

def getProbablePrime(bitcount):
    while True:
        p = random.randint(2**(bitcount - 1), 2**bitcount - 1)
        if not hasSmallPrimeFactor(p) and isProbablePrime(p, 5):
            return p

def genKey(keysize):
    e = 3
    bitcount = (keysize + 1) // 2 + 1

    p = 7
    while (p - 1) % e == 0:
        p = getProbablePrime(bitcount)

    q = p
    while q == p or (q - 1) % e == 0:
        q = getProbablePrime(bitcount)

    n = p * q
    et = (p - 1) * (q - 1)
    d = invmod(e, et)
    pub = (e, n)
    priv = (d, n)
    return (pub, priv)

def encryptnum(pub, m):
    (e, n) = pub
    if m < 0 or m >= n:
        raise ValueError(str(m) + ' out of range')
    return pow(m, e, n)

def decryptnum(priv, c):
    (d, n) = priv
    if c < 0 or c >= n:
        raise ValueError(str(c) + ' out of range')
    return pow(c, d, n)

# Drops leading zero bytes.
def bytestonum(s):
    return int(s.encode('hex'), 16) #int.from_bytes(s, byteorder='big')

def numtobytes(k):
    return to_bytes(k, (k.bit_length() + 7) // 8)

def encryptbytes(pub, mbytes):
    m = bytestonum(mbytes)
    c = encryptnum(pub, m)
    cbytes = numtobytes(c)
    return cbytes

def decryptbytes(priv, cbytes):
    c = bytestonum(cbytes)
    m = decryptnum(priv, c)
    mstr = numtobytes(m)
    return mstr


pub, priv = genKey(128)
m = b'test'
c = encryptbytes(pub, m)
m2 = decryptbytes(priv, c)
if m != m2:
    raise Exception(str(m) + ' != ' + str(m2))

### Challenge 40 - Implement an E=3 RSA Broadcast attack

In [4]:
import binascii
import math
from __future__ import division

(pub0, priv0) = genKey(256)
(pub1, priv1) = genKey(256)
(pub2, priv2) = genKey(256)

plaintext = b'This is a plaintext string'
plainnum = bytestonum(plaintext)

c0 =  encryptnum(pub0, plainnum)
c1 =  encryptnum(pub1, plainnum)
c2 =  encryptnum(pub2, plainnum)

n0 = pub0[1]
n1 = pub1[1]
n2 = pub2[1]

ms0 = n1 * n2
ms1 = n0 * n2
ms2 = n0 * n1

N = n0 * n1 * n2

r0 = (c0 * ms0 * invmod(ms0, n0))
r1 = (c1 * ms1 * invmod(ms1, n1))
r2 = (c2 * ms2 * invmod(ms2, n2))

r = (r0 + r1 + r2) % N

def floorRoot(n, s):
    b = n.bit_length()
    p = math.ceil(b/s)
    p = int(p)
    x = 2**p
    while x > 1:
        y = (((s - 1) * x) + (n // (x**(s-1)))) // s
        if y >= x:
            return x
        x = y
    return 1

m = floorRoot(r, 3)
mstr = numtobytes(m)

if mstr != plaintext:
    raise Exception(mstr + b' != ' + plaintext)

剩下的部分是man in the middle 的server練習 有興趣的可以嘗試!!

# Set6
### Challenge 41 -  Implement unpadded message recovery oracle

想像一個 web application， 用JS來做encryption， 以RSA來加密且沒有經過padding。

你可以傳送任何RSA區塊，server會傳回plaintext，但不能傳送相同訊息兩次，我們假設server保存先前的hash一段時間且message 內嵌有timestamp:

```
{
  time: 1356304276,
  social: '555-55-5555',
}
```

你可以capture其他人的message丟給server但server會reject!

我們攻擊法是，

1. 得到 ciphertext C
2. N 和 E 分別為 public modulus 和 exponent
3. 假設 S 為隨機數 > 1 mod N. 

現在:
```
C' = ((S**E mod N) C) mod N
```

傳送 C'(和 C完全不同) 給 server, 解出 P', 也和 P完全不同

但是...

```
          P'
    P = -----  mod N
          S
```

Oops!

In [5]:
from Crypto.Random import random
import binascii
import hashlib

decryptedHashes = set()

pub, priv = genKey(128)

def encrypt(plaintext):
    return encryptbytes(pub, plaintext)

def decryptOnce(ciphertext):
    sha1 = hashlib.sha1()
    sha1.update(ciphertext)
    digest = sha1.digest()
    if digest in decryptedHashes:
        raise ValueError('Already decrypted')
    decryptedHashes.add(digest)
    return decryptbytes(priv, ciphertext)


plaintext = b'secret text'
ciphertext = encrypt(plaintext)
plaintext2 = decryptOnce(ciphertext)
if plaintext2 != plaintext:
    raise ValueError(plaintext2 + b' != ' + plaintext)

(e, n) = pub
s = random.randint(2, n - 1)
c = bytestonum(ciphertext)
c2 = (pow(s, e, n) * c) % n
ciphertext2 = numtobytes(c2)
plaintext3 = decryptOnce(ciphertext2)
p3 = bytestonum(plaintext3)
p4 = (p3 * invmod(s, n)) % n
plaintext4 = numtobytes(p4)

if plaintext4 != plaintext:
    raise ValueError(plaintext4 + b' != ' + plaintext)


### Challenge 42 - Bleichenbacher's e=3 RSA Attack
曾經破解FireFox的TLS!

RSA 用 3當作 encrypting exponent 非常普遍(速度快)

```
 c = m ** 3 % n
```

e=3 是安全的。但使用 low-exponent RSA 若 message blocks不夠大到在3次方後能超過 modulus 就會有問題，例如 The block 00:02 (imagine sufficient zero-padding) 用 e=3 RSA; 得到 00:08.

當 RSA 用來簽章， 情況變成 verifier "decrypts" message 藉由3次方. 這將產生 "plaintext" 來驗證其合法性!

當你用RSA簽章時，你提供digest PKCS1.5 standard formats 為:

```
00h 01h ffh ffh ... ffh ffh 00h ASN.1 GOOP HASH
```

這將產生 "right-justified" hash (the last byte of the hash is the last byte of the message)。

數年前一個常見的  RSA verifiers flaw為: 他們先 "decrypting" (cubing them modulo the public exponent) 然後 "parsing" 它們尋找 00h 01h ... ffh 00h ASN.1 HASH.

但當 verifier 不完全check padding時將產生偽造的 signature.

如何做呢?
找出一個數字當 cubed (a) 不超過modulus (thus bypassing the key entirely) 且 (b)產生 block that starts "00h 01h ffh ... 00h ASN.1 HASH".

試著 Forge a 1024-bit RSA signature (針對 string "hi mom"). 確認你的encryption能接受這個signature!

In [6]:
import hashlib
import re

def generateSignature(priv, message):
    sha1 = hashlib.sha1()
    sha1.update(message)
    digest = sha1.digest()
    block = b'\x00\x01' + (b'\xff' * (128 - len(digest) - 3)) + b'\x00' + digest
    signature = decryptbytes(priv, block)
    return signature

def verifySignature(pub, message, signature):
    block = b'\x00' + encryptbytes(pub, signature)
    r = re.compile(b'\x00\x01\xff+?\x00(.{20})', re.DOTALL)
    m = r.match(block)
    if not m:
        return False
    digest = m.group(1)
    sha1 = hashlib.sha1()
    sha1.update(message)
    return digest == sha1.digest()

def forgeSignature(pub, message):
    sha1 = hashlib.sha1()
    sha1.update(message)
    digest = sha1.digest()
    block = b'\x00\x01\xff\x00' + digest + (b'\x00' * (128 - len(digest) - 4))
    (d, n) = pub
    blocknum = bytestonum(block)
    signaturenum = floorRoot(blocknum, 3) + 1
    signature = numtobytes(signaturenum)
    return signature


message = b'hi mom'
pub, priv = genKey(1024)
signature = generateSignature(priv, message)
if not verifySignature(pub, message, signature):
    raise Exception(message + b' has invalid signature ' + signature)
forgedSignature = forgeSignature(pub, message)
if not verifySignature(pub, message, forgedSignature):
    raise Exception(message + b' has invalid signature ' + forgedSignature)

### Challenge 43 -  DSA key recovery from nonce

第一步: 重定準備工作

第二步: 實現DSA，包含 signing, verifying, 和 parameter generation.

如果不想要實現 parameter generation 可用:
```
 p = 800000000000000089e1855218a0e7dac38136ffafa72eda7
     859f2171e25e65eac698c1702578b07dc2a1076da241c76c6
     2d374d8389ea5aeffd3226a0530cc565f3bf6b50929139ebe
     ac04f48c3c84afb796d61e5a4f9a8fda812ab59494232c7d2
     b4deb50aa18ee9e132bfa85ac4374d7f9091abc3d015efc87
     1a584471bb1
 
 q = f4f47f05794b256174bba6e9b396a7707e563c5b
 
 g = 5958c9d3898b224b12672c0b98e06c60df923cb8bc999d119
     458fef538b8fa4046c8db53039db620c094c9fa077ef389b5
     322a559946a71903f990f1f7e0e025e2d7f7cf494aff1a047
     0f5b64c36b625a097f1651fe775323556fe00b3608c887892
     878480e99041be601a62166ca6894bdd41a7054ec89f756ba
     9fc95302291
```

DSA signing 會產生一個隨機的 subkey "k". 

這是第一個關於DSA subkey的 challenge

給定 k 要來 recover DSA private key x

```
     (s * k) - H(msg)
x = ----------------  mod q
            r
```

試著做幾次以上步驟，然後假定給定以上parameter，產生了keypair pubkey為

```
y = 84ad4719d044495496a3201c8ff484feb45b962e7302e56a392aee4
    abab3e4bdebf2955b4736012f21a08084056b19bcd7fee56048e004
    e44984e2f411788efdc837a0d2e5abb7b555039fd243ac01f0fb2ed
    1dec568280ce678e931868d23eb095fde9d3779191b8c0299d6e07b
    bb283e6633451e535c45513b2d33c99ea17
```

然後簽章

```
For those that envy a MC it can be hazardous to your health
So be friendly, a matter of life and death, just like a etch-a-sketch
```

SHA1為 ``d2d0714f014a9784047eaeccf956520045c45265``轉成integer: ``0xd2d0714f014a9784047eaeccf956520045c45265).``

得到

```
r = 548099063082341131477253921760299949438196259240
s = 857042759984254168557880549501802188789837994940
```

SHA1 值為

```
0954edd5e0afe5542a4adf012611a91912a3ec16
```


In [7]:
from Crypto.Random import random
import binascii
import hashlib

def genP(L, q):
    minK = (2**(L-1) + q-1)//q
    maxK = (2**L - 1)//q
    while True:
        k = random.randint(minK, maxK)
        p = k*q + 1
        if isProbablePrime(p, 5):
            return (k, p)

def genG(p, q, k):
    for h in range(2, p - 1):
        g = pow(h, k, p)
        if g != 1:
            return g
    raise Exception('unexpected')

def genParams(L, N):
    q = getProbablePrime(N)
    k, p = genP(L, q)
    g = genG(p, q, k)
    return (p, q, g)

def areValidParams(L, N, p, q, g):
    return ((q.bit_length() == N) and isProbablePrime(q, 5) and (p.bit_length() == L) and isProbablePrime(p, 5) and ((p-1) % q == 0) and pow(g, q, p) == 1)

def genKeys(p, q, g):
    x = random.randint(1, q-1)
    y = pow(g, x, p)
    pub = (p, q, g, y)
    priv = x
    return (pub, priv)

def areValidKeys(pub, priv):
    (p, q, g, y) = pub
    x = priv
    return y == pow(g, x, p)

def hash(message):
    sha1 = hashlib.sha1()
    sha1.update(message)
    digest = sha1.digest()
    return int(digest.encode('hex'), 16)

def signHashWithK(H, pub, priv, k):
    (p, q, g, y) = pub
    x = priv
    r = pow(g, k, p) % q
    if r == 0:
        return None
    kInv = invmod(k, q)
    s = (kInv * (H + x * r)) % q
    if s == 0:
        return None
    return (r, s)

def signHash(H, pub, priv):
    (_, q, _, _) = pub
    while True:
        k = random.randint(1, q-1)
        signature = signHashWithK(H, pub, priv, k)
        if not signature:
            continue
        return signature

def sign(message, pub, priv):
    return signHash(hash(message), pub, priv)

def verifySignatureHash(H, signature, pub):
    (r, s) = signature
    (p, q, g, y) = pub
    if r <= 0 or r >= q or s <= 0 or s >= q:
        return False
    w = invmod(s, q)
    u1 = (H * w) % q
    u2 = (r * w) % q
    v = ((pow(g, u1, p) * pow(y, u2, p)) % p) % q
    return v == r

def verifySignature(message, signature, pub):
    return verifySignatureHash(hash(message), signature, pub)

def extractKey(H, r, s, k, pub):
    (p, q, g, y) = pub
    rInv = invmod(r, q)
    return (rInv * (s * k - H)) % q

def bruteForceKey(H, r, s, pub, kMin, kMax):
    for k in range(kMin, kMax):
        priv = extractKey(H, r, s, k, pub)
        if areValidKeys(pub, priv):
            return (k, priv)
    return None


L = 1024
N = 160
(p, q, g) = (0x800000000000000089e1855218a0e7dac38136ffafa72eda7859f2171e25e65eac698c1702578b07dc2a1076da241c76c62d374d8389ea5aeffd3226a0530cc565f3bf6b50929139ebeac04f48c3c84afb796d61e5a4f9a8fda812ab59494232c7d2b4deb50aa18ee9e132bfa85ac4374d7f9091abc3d015efc871a584471bb1, 
             0xf4f47f05794b256174bba6e9b396a7707e563c5b, 
             0x5958c9d3898b224b12672c0b98e06c60df923cb8bc999d119458fef538b8fa4046c8db53039db620c094c9fa077ef389b5322a559946a71903f990f1f7e0e025e2d7f7cf494aff1a0470f5b64c36b625a097f1651fe775323556fe00b3608c887892878480e99041be601a62166ca6894bdd41a7054ec89f756ba9fc95302291)

message = b'''For those that envy a MC it can be hazardous to your health
So be friendly, a matter of life and death, just like a etch-a-sketch
'''
expectedH = 0xd2d0714f014a9784047eaeccf956520045c45265

H = hash(message)
if H != expectedH:
    raise Exception(hex(H) + ' != ' + hex(expectedH))
y = 0x84ad4719d044495496a3201c8ff484feb45b962e7302e56a392aee4abab3e4bdebf2955b4736012f21a08084056b19bcd7fee56048e004e44984e2f411788efdc837a0d2e5abb7b555039fd243ac01f0fb2ed1dec568280ce678e931868d23eb095fde9d3779191b8c0299d6e07bbb283e6633451e535c45513b2d33c99ea17
r = 548099063082341131477253921760299949438196259240
s = 857042759984254168557880549501802188789837994940

pub = (p, q, g, y)
if not verifySignatureHash(H, (r, s), pub):
    raise Exception('unexpected')
    
k, priv = bruteForceKey(H, r, s, pub, 0, 2**16)
expectedHashPriv = 0x0954edd5e0afe5542a4adf012611a91912a3ec16
print expectedHashPriv
print hex(priv)
print hex(priv)[2:]
print hex(priv)[2:].encode('ascii')
hashPriv = hash(hex(priv)[2:].encode('ascii'))
print hashPriv

if hashPriv != expectedHashPriv:
    raise Exception(str(hashPriv) + ' != ' + str(expectedHashPriv))
    
(r2, s2) = signHashWithK(H, pub, priv, k)
if r2 != r:
    raise Exception(str(r2) + ' != ' + str(r))
if s2 != s:
    raise Exception(str(s2) + ' != ' + str(s))
print(k, priv)

53274897924661483746453784626298389645641378838
0x15fb2873d16b3e129ff76d0918fd7ada54659e49L
15fb2873d16b3e129ff76d0918fd7ada54659e49L
15fb2873d16b3e129ff76d0918fd7ada54659e49L
918934021177755966061404806803245150683580089139


Exception: 918934021177755966061404806803245150683580089139 != 53274897924661483746453784626298389645641378838

### Challenge 44 - DSA nonce recovery from repeated nonce

Break PS3!

In [8]:
import itertools
import re

def getMessage(messageLines):
    msg = re.match('^msg: (.*)', messageLines[0]).group(1).encode('ascii')
    m = int(re.match('^m: (.*)', messageLines[3]).group(1), 16)
    H = hash(msg)
    if m != H:
        raise Exception(hex(H) + ' != ' + hex(m))
    s = int(re.match('^s: (.*)', messageLines[1]).group(1))
    r = int(re.match('^r: (.*)', messageLines[2]).group(1))
    return (msg, s, r, m)

def getMessages():
    lines = list(open('Text/44.txt', 'r').readlines())
    messageLines = [lines[4*i:4*i+4] for i in range(len(lines) // 4)]
    return [getMessage(x) for x in messageLines]

def checkForCommonK(pub, msg1, msg2):
    (p, q, g, y) = pub
    (_, s1, r1, m1) = msg1
    (_, s2, r2, m2) = msg2
    ds = (s1 - s2) % q
    dm = (m1 - m2) % q
    dsInv = invmod(ds, q)
    k = (dm * dsInv) % q
    priv1 = extractKey(m1, r1, s1, k, pub)
    priv2 = extractKey(m2, r2, s2, k, pub)
    if priv1 == priv2 and areValidKeys(pub, priv1) and areValidKeys(pub, priv2):
        return (k, priv1)
    return (None, None)

def breakRepeatedK(pub, messages):
    for pr in itertools.combinations(messages, 2):
        (msg1, msg2) = pr
        (k, priv) = checkForCommonK(pub, msg1, msg2)
        if k:
            return (k, priv)
    return (None, None)


(p, q, g) = (0x800000000000000089e1855218a0e7dac38136ffafa72eda7859f2171e25e65eac698c1702578b07dc2a1076da241c76c62d374d8389ea5aeffd3226a0530cc565f3bf6b50929139ebeac04f48c3c84afb796d61e5a4f9a8fda812ab59494232c7d2b4deb50aa18ee9e132bfa85ac4374d7f9091abc3d015efc871a584471bb1, 0xf4f47f05794b256174bba6e9b396a7707e563c5b, 0x5958c9d3898b224b12672c0b98e06c60df923cb8bc999d119458fef538b8fa4046c8db53039db620c094c9fa077ef389b5322a559946a71903f990f1f7e0e025e2d7f7cf494aff1a0470f5b64c36b625a097f1651fe775323556fe00b3608c887892878480e99041be601a62166ca6894bdd41a7054ec89f756ba9fc95302291)
y = 0x2d026f4bf30195ede3a088da85e398ef869611d0f68f0713d51c9c1a3a26c95105d915e2d8cdf26d056b86b8a7b85519b1c23cc3ecdc6062650462e3063bd179c2a6581519f674a61f1d89a1fff27171ebc1b93d4dc57bceb7ae2430f98a6a4d83d8279ee65d71c1203d2c96d65ebbf7cce9d32971c3de5084cce04a2e147821
messages = getMessages()
pub = (p, q, g, y)
k, priv = breakRepeatedK(pub, messages)
expectedHashPriv = 0xca8f6f7c66fa362d40760d135b763eb8527d3d52
hashPriv = hash(hex(priv)[2:].encode('ascii'))
if hashPriv != expectedHashPriv:
    raise Exception(str(hashPriv) + ' != ' + str(expectedHashPriv))
for msg in messages:
    (_, s, r, m) = msg
    if not verifySignatureHash(m, (r, s), pub):
        raise Exception('unexpected')

Exception: 1157803233552576263286851715937593123001740977645 != 1156414854065437815208112707638019467062269525330

### Challenge 45 - DSA parameter tampering

Take your DSA code from the previous exercise. Imagine it as part of an algorithm in which the client was allowed to propose domain parameters (the p and q moduli, and the g generator).

This would be bad, because attackers could trick victims into accepting bad parameters. Vaudenay gave two examples of bad generator parameters: generators that were 0 mod p, and generators that were 1 mod p.

Use the parameters from the previous exercise, but substitute 0 for "g". Generate a signature. You will notice something bad. Verify the signature. Now verify any other signature, for any other string.

Now, try (p+1) as "g". With this "g", you can generate a magic signature s, r for any DSA public key that will validate against any string. For arbitrary z:

  r = ((y**z) % p) % q

        r
  s =  --- % q
        z
Sign "Hello, world". And "Goodbye, world".

In [9]:
from Crypto.Random import random

# Doesn't check for 0 values for r and s.
def relaxedSign(message, pub, priv):
    H = hash(message)
    (p, q, g, y) = pub
    k = random.randint(1, q-1)
    x = priv
    r = pow(g, k, p) % q
    kInv = invmod(k, q)
    s = (kInv * (H + x * r)) % q
    return (r, s)

# Doesn't check for 0 values for r and s.
def relaxedVerifySignature(message, signature, pub):
    H = hash(message)
    (r, s) = signature
    (p, q, g, y) = pub
    if r < 0 or r >= q or s < 0 or s >= q:
        return False
    w = invmod(s, q)
    u1 = (H * w) % q
    u2 = (r * w) % q
    v = ((pow(g, u1, p) * pow(y, u2, p)) % p) % q
    return v == r


(p, q, g) = (0x800000000000000089e1855218a0e7dac38136ffafa72eda7859f2171e25e65eac698c1702578b07dc2a1076da241c76c62d374d8389ea5aeffd3226a0530cc565f3bf6b50929139ebeac04f48c3c84afb796d61e5a4f9a8fda812ab59494232c7d2b4deb50aa18ee9e132bfa85ac4374d7f9091abc3d015efc871a584471bb1, 0xf4f47f05794b256174bba6e9b396a7707e563c5b, 0x5958c9d3898b224b12672c0b98e06c60df923cb8bc999d119458fef538b8fa4046c8db53039db620c094c9fa077ef389b5322a559946a71903f990f1f7e0e025e2d7f7cf494aff1a0470f5b64c36b625a097f1651fe775323556fe00b3608c887892878480e99041be601a62166ca6894bdd41a7054ec89f756ba9fc95302291)
pub, priv = genKeys(p, q, 0)
message1 = b'Hello, world'
signature1 = relaxedSign(message1, pub, priv)
message2 = b'Goodbye, world'
signature2 = relaxedSign(message2, pub, priv)
print(message1, signature1, relaxedVerifySignature(message1, signature1, pub))
print(message2, signature2, relaxedVerifySignature(message2, signature2, pub))
print(relaxedVerifySignature(message1, signature2, pub))
print(relaxedVerifySignature(message2, signature1, pub))

pub, priv = genKeys(p, q, p+1)
(_, _, _, y) = pub
z = 2
invZ = invmod(2, q)
r = ((y**z) % p) % q
s = (r * invZ) % q
signature = (r, s)
print(signature)
print(message1, verifySignature(message1, signature, pub))
print(message2, verifySignature(message2, signature, pub))

('Hello, world', (0L, 397549351888133772335310194473860637229026712484L), True)
('Goodbye, world', (0L, 1225952482815394023852979553550677454237538630564L), True)
True
True
(1L, 699223097516205126020108705086851195054347460142L)
('Hello, world', True)
('Goodbye, world', True)


### Challenge 46 - RSA parity oracle

When does this ever happen?
This is a bit of a toy problem, but it's very helpful for understanding what RSA is doing (and also for why pure number-theoretic encryption is terrifying). Trust us, you want to do this before trying the next challenge. Also, it's fun.
Generate a 1024 bit RSA key pair.

Write an oracle function that uses the private key to answer the question "is the plaintext of this message even or odd" (is the last bit of the message 0 or 1). Imagine for instance a server that accepted RSA-encrypted messages and checked the parity of their decryption to validate them, and spat out an error if they were of the wrong parity.

Anyways: function returning true or false based on whether the decrypted plaintext was even or odd, and nothing else.

Take the following string and un-Base64 it in your code (without looking at it!) and encrypt it to the public key, creating a ciphertext:

VGhhdCdzIHdoeSBJIGZvdW5kIHlvdSBkb24ndCBwbGF5IGFyb3VuZCB3aXRoIHRoZSBGdW5reSBDb2xkIE1lZGluYQ==
With your oracle function, you can trivially decrypt the message.

Here's why:

RSA ciphertexts are just numbers. You can do trivial math on them. You can for instance multiply a ciphertext by the RSA-encryption of another number; the corresponding plaintext will be the product of those two numbers.
If you double a ciphertext (multiply it by (2**e)%n), the resulting plaintext will (obviously) be either even or odd.
If the plaintext after doubling is even, doubling the plaintext didn't wrap the modulus --- the modulus is a prime number. That means the plaintext is less than half the modulus.
You can repeatedly apply this heuristic, once per bit of the message, checking your oracle function each time.

Your decryption function starts with bounds for the plaintext of [0,n].

Each iteration of the decryption cuts the bounds in half; either the upper bound is reduced by half, or the lower bound is.

After log2(n) iterations, you have the decryption of the message.

Print the upper bound of the message as a string at each iteration; you'll see the message decrypt "hollywood style".

Decrypt the string (after encrypting it to a hidden private key) above.

In [10]:
import base64
from __future__ import division

pub, priv = genKey(1024)

def parityOracle(c):
    p = decryptnum(priv, c)
    return p % 2

def deducePlaintext(ciphertext, pub, parityOracle):
    (e, n) = pub
    low = 0
    high = 1
    denom = 1
    c = bytestonum(ciphertext)
    k = pow(2, e, n)
    for _ in range(n.bit_length()):
        c  = (c * k) % n
        p = parityOracle(c)
        d = high - low
        low *= 2
        high *= 2
        denom *= 2
        if p == 0:
            high -= d
        else:
            low += d
        hightext = numtobytes(n * high // denom)
        print(hightext)
    return hightext


encodedPlaintext = b'VGhhdCdzIHdoeSBJIGZvdW5kIHlvdSBkb24ndCBwbGF5IGFyb3VuZCB3aXRoIHRoZSBGdW5reSBDb2xkIE1lZGluYQ=='
plaintext = base64.b64decode(encodedPlaintext)
ciphertext = encryptbytes(pub, plaintext)
plaintext2 = deducePlaintext(ciphertext, pub, parityOracle)
if plaintext2 != plaintext:
    raise Exception(b'Invalid plaintext ' + plaintext2)

��c�K�e/W4��Vp�
�R���|Gq]C<7a_V� �A5�{��E������/ٿ����^L&[�E�o�P�>~A��h7q�x�7��Y�|����?�#v����36�1�� ;n�p�np��%�2���}b+8f
w�\U�>#��������q[ n���ʽ�]�̓x�׊Qf���ܗ���l��֯&-�"��(p�? ��4���wǛ�r���Sw��t���;�[��X��{p���
�e�
�F78X��K��>��3
�z�;���V{�)��q:O�ǝ����ͬD�A��o�HۅC��A��,~	t���X���
�W{ψ�+�g�� ���V��&���t�h�`�5��Y��w%�7��8�u�Ʉ���-�
�=o���ܫ=���8�'�c���V�Ff�"F ��7äm¡�n�э�?�R�sO�Eg�
�QYW�K�Y�o��J,��޻����͜q^����e��^�����w����y�U���n��N��1�o��x#3k#on����P�Ph���])z���"��`
���p�v[��4cs���.���\��Y�0
�J�y��u�݄}[��u�VU��lƶ�R�7�w��f��gW��y0�o����C�������ǭ�<�{�g��'����y�������H�ۆ�t��T;-�T1����@�^�i����
��c�K�e/W4��Vp�
�R���|Gq]C<7a_V� �A5�{��E������/ٿ����^L&[�E�o�P�>~A��h7q�x�7��Y�|����?�#v����36�1�� ;n�p�np��%�2���}b+8f
w�\U�>#��������q[ n���ʽ�]�̓x�׊Qf���ܗ���l��֯&-�"��(p�? ��4���wǛ�r���Sw��t���;�[��X��{p���
�e�
�F78X��K��>��3
�z�;���V{�)��q:O�ǝ����ͬD�A��o�HۅC��A��,~	t���X��
�W{ψ�+�g��

### Challenge 47 - Bleichenbacher's PKCS 1.5 Padding Oracle

魔王降臨!!

Degree of difficulty: moderate
These next two challenges are the hardest in the entire set.
Let us Google this for you: "Chosen ciphertext attacks against protocols based on the RSA encryption standard"

This is Bleichenbacher from CRYPTO '98; I get a bunch of .ps versions on the first search page.

Read the paper. It describes a padding oracle attack on PKCS#1v1.5. The attack is similar in spirit to the CBC padding oracle you built earlier; it's an "adaptive chosen ciphertext attack", which means you start with a valid ciphertext and repeatedly corrupt it, bouncing the adulterated ciphertexts off the target to learn things about the original.

This is a common flaw even in modern cryptosystems that use RSA.

It's also the most fun you can have building a crypto attack. It involves 9th grade math, but also has you implementing an algorithm that is complex on par with finding a minimum cost spanning tree.

The setup:

Build an oracle function, just like you did in the last exercise, but have it check for plaintext[0] == 0 and plaintext[1] == 2.
Generate a 256 bit keypair (that is, p and q will each be 128 bit primes), [n, e, d].
Plug d and n into your oracle function.
PKCS1.5-pad a short message, like "kick it, CC", and call it "m". Encrypt to to get "c".
Decrypt "c" using your padding oracle.
For this challenge, we've used an untenably small RSA modulus (you could factor this keypair instantly). That's because this exercise targets a specific step in the Bleichenbacher paper --- Step 2c, which implements a fast, nearly O(log n) search for the plaintext.

Things you want to keep in mind as you read the paper:

RSA ciphertexts are just numbers.
RSA is "homomorphic" with respect to multiplication, which means you can multiply c * RSA(2) to get a c' that will decrypt to plaintext * 2. This is mindbending but easy to see if you play with it in code --- try multiplying ciphertexts with the RSA encryptions of numbers so you know you grok it.
What you need to grok for this challenge is that Bleichenbacher uses multiplication on ciphertexts the way the CBC oracle uses XORs of random blocks.
A PKCS#1v1.5 conformant plaintext, one that starts with 00:02, must be a number between 02:00:00...00 and 02:FF:FF..FF --- in other words, 2B and 3B-1, where B is the bit size of the modulus minus the first 16 bits. When you see 2B and 3B, that's the idea the paper is playing with.
To decrypt "c", you'll need Step 2a from the paper (the search for the first "s" that, when encrypted and multiplied with the ciphertext, produces a conformant plaintext), Step 2c, the fast O(log n) search, and Step 3.

Your Step 3 code is probably not going to need to handle multiple ranges.

We recommend you just use the raw math from paper (check, check, double check your translation to code) and not spend too much time trying to grok how the math works.


### Challenge 48

Cryptanalytic MVP award
This is an extraordinarily useful attack. PKCS#1v15 padding, despite being totally insecure, is the default padding used by RSA implementations. The OAEP standard that replaces it is not widely implemented. This attack routinely breaks SSL/TLS.
This is a continuation of challenge #47; it implements the complete BB'98 attack.

Set yourself up the way you did in #47, but this time generate a 768 bit modulus.

To make the attack work with a realistic RSA keypair, you need to reproduce step 2b from the paper, and your implementation of Step 3 needs to handle multiple ranges.

The full Bleichenbacher attack works basically like this:

Starting from the smallest 's' that could possibly produce a plaintext bigger than 2B, iteratively search for an 's' that produces a conformant plaintext.
For our known 's1' and 'n', solve m1=m0s1-rn (again: just a definition of modular multiplication) for 'r', the number of times we've wrapped the modulus.
'm0' and 'm1' are unknowns, but we know both are conformant PKCS#1v1.5 plaintexts, and so are between [2B,3B].
We substitute the known bounds for both, leaving only 'r' free, and solve for a range of possible 'r' values. This range should be small!
Solve m1=m0s1-rn again but this time for 'm0', plugging in each value of 'r' we generated in the last step. This gives us new intervals to work with. Rule out any interval that is outside 2B,3B.
Repeat the process for successively higher values of 's'. Eventually, this process will get us down to just one interval, whereupon we're back to exercise #47.
What happens when we get down to one interval is, we stop blindly incrementing 's'; instead, we start rapidly growing 'r' and backing it out to 's' values by solving m1=m0s1-rn for 's' instead of 'r' or 'm0'. So much algebra! Make your teenage son do it for you! *Note: does not work well in practice*

In [11]:
from Crypto.Random import random
import re
from __future__ import division

pub, priv = genKey(256)

def parityOracle(c):
    _, n = pub
    k = (n.bit_length() + 7) // 8
    p = decryptnum(priv, c)
    pbytes = numtobytes(p)
    pbytes = (b'\x00' * (k - len(pbytes))) + pbytes
    return pbytes[0:2] == b'\x00\x02'

def randnonzerobytes(k):
    return bytes(random.sample(range(1, 256), k))

def padPKCS15(s, n):
    if len(s) < 8:
        raise Exception('unexpected')
    k = (n.bit_length() + 7) // 8
    padding = randnonzerobytes(k - 3 - len(s))
    return b'\x00\x02' + padding + b'\x00' + s

def computeFirstS(e, n, B, c0, parityOracle):
    s = (n + 3*B - 1) // (3*B)
    while True:
        c = (c0 * pow(s, e, n)) % n
        if parityOracle(c):
            return (s, c)
        s += 1

def computeNextS(e, n, M, s, B, c0):
    if len(M) > 1:
        raise Exception('unexpected')
    else:
        a, b = M[0]
        r = (2 * (b * s - 2 * B) + n - 1) // n
        while True:
            sLow = (2*B + r*n + b - 1) // b
            sHigh = (3*B + r*n + a - 1) // a
            for s in range(sLow, sHigh):
                c = (c0 * pow(s, e, n)) % n
                if parityOracle(c):
                    return (s, c)
            r += 1

def getNextInterval(n, M, s, B):
    if len(M) > 1:
        raise Exception('unexpected')
    a, b = M[0]
    minR = (a * s - 3 * B + 1 + n - 1) // n
    maxR = (b * s - 2 * B) // n
    if maxR > minR:
        raise Exception('unexpected')
    r = minR
    ai = max(a, (2*B + r*n + s - 1) // s)
    bi = min(b, (3*B - 1 + r*n) // s)
    if ai > bi:
        raise Exception('unexpected')
    return [(ai, bi)]

def deducePlaintext(ciphertext, pub, parityOracle):
    e, n = pub
    k = (n.bit_length() + 7) // 8
    B = 2**(8*(k-2))
    c0 = bytestonum(ciphertext)
    M = [(2*B, 3*B - 1)]
    (s, c) = computeFirstS(e, n, B, c0, parityOracle)
    M = getNextInterval(n, M, s, B)
    while True:
        if len(M) == 1 and M[0][0] == M[0][1]:
            m = M[0][0]
            return b'\x00' + numtobytes(m)
        (s, c) = computeNextS(e, n, M, s, B, c0)
        M = getNextInterval(n, M, s, B)


_, n = pub
plaintext = padPKCS15(b'kick it, CC', n)
ciphertext = encryptbytes(pub, plaintext)
plaintext2 = deducePlaintext(ciphertext, pub, parityOracle)
if plaintext2 != plaintext:
    raise Exception(plaintext2 + b' != ' + plaintext)

ValueError: 4022223726942804016231672067225580976132849401592327372432046767382602490793950594866770570653122507834722149984382362079711758490017511906856678773470818403842017932468123741241945236523432208383777880450291491825433976457428267680080460792643 out of range

In [ ]:
from __future__ import division

16/5

In [ ]:
16/5

This is an extraordinarily useful attack. PKCS#1v15 padding, despite being totally insecure, is the default padding used by RSA implementations. The OAEP standard that replaces it is not widely implemented. This attack routinely breaks SSL/TLS.

In [12]:
import itertools
import re

pub, priv = genKey(768)

def parityOracle(c):
    _, n = pub
    k = (n.bit_length() + 7) // 8
    p = decryptnum(priv, c)
    pbytes = numtobytes(p)
    pbytes = (b'\x00' * (k - len(pbytes))) + pbytes
    return pbytes[0:2] == b'\x00\x02'

def computeNextS(e, n, M, s, B, c0):
    if len(M) > 1:
        while True:
            s += 1
            c = (c0 * pow(s, e, n)) % n
            if parityOracle(c):
                return (s, c)
    else:
        a, b = M[0]
        r = (2 * (b * s - 2 * B) + n - 1) // n
        while True:
            sLow = (2*B + r*n + b - 1) // b
            sHigh = (3*B + r*n + a - 1) // a
            for s in range(sLow, sHigh):
                c = (c0 * pow(s, e, n)) % n
                if parityOracle(c):
                    return (s, c)
            r += 1

def getNextInterval(n, M, s, B):
    if len(M) == 0:
        raise Exception('unexpected1')
    Mnew = []
    for a, b in M:
        minR = (a * s - 3 * B + 1 + n - 1) // n
        maxR = (b * s - 2 * B) // n
        for r in range(minR, maxR + 1):
            ai = max(a, (2*B + r*n + s - 1) // s)
            bi = min(b, (3*B - 1 + r*n) // s)
            if ai > bi:
                continue
            Mnew += [(ai, bi)]
    if len(Mnew) == 0:
        raise Exception('unexpected')
    return Mnew

def deducePlaintext(ciphertext, pub, parityOracle):
    e, n = pub
    k = (n.bit_length() + 7) // 8
    B = 2**(8*(k-2))
    c0 = bytestonum(ciphertext)
    M = [(2*B, 3*B - 1)]
    (s, c) = computeFirstS(e, n, B, c0, parityOracle)
    M = getNextInterval(n, M, s, B)
    while True:
        if len(M) == 1 and M[0][0] == M[0][1]:
            m = M[0][0]
            return b'\x00' + numtobytes(m)
        (s, c) = computeNextS(e, n, M, s, B, c0)
        M = getNextInterval(n, M, s, B)

if __name__ == '__main__':
    _, n = pub
    plaintext = padPKCS15(b'kick it, CC', n)
    ciphertext = encryptbytes(pub, plaintext)
    plaintext2 = deducePlaintext(ciphertext, pub, parityOracle)
    if plaintext2 != plaintext:
        raise Exception(plaintext2 + b' != ' + plaintext)

ValueError: 15051282195972673793758269939745559348974287280548229092546769196421524457731618815106064319356963875062919215155680298433413316698614214656170537292710886838150686632438385112689454385974872470908340845613697282903199171115051038528160430929657135946594194667218094832753335820202747066537276939189031327711600716039548021902308011143216334587773732477846802844631180695787824024248955336648727556048470414920610450827071074055546550157277762303500998173654293043845133083499045827446225778556751643712910977564841996397266104869671567510338694958771155746126606933217164989364144742285599965782017410711915968648234682235094919898661349853560412478419848646815297536757085293028947064669789294413913376525869134889396579293451782761315725696927538249679925265530219015367767168384013885891198350249797317179450964052897593857173000138503039547799292186890201255342809346614576223484628609445417793251432497371720837630916622119514161987 out of range

# Set 7
### Challeneg 49 - CBC-MAC Message Forgery
Let's talk about CBC-MAC.

CBC-MAC is like this:

Take the plaintext P.
Encrypt P under CBC with key K, yielding ciphertext C.
Chuck all of C but the last block C[n].
C[n] is the MAC.
Suppose there's an online banking application, and it carries out user requests by talking to an API server over the network. Each request looks like this:

message || IV || MAC
The message looks like this:

from=#{from_id}&to=#{to_id}&amount=#{amount}
Now, write an API server and a web frontend for it. (NOTE: No need to get ambitious and write actual servers and web apps. Totally fine to go lo-fi on this one.) The client and server should share a secret key K to sign and verify messages.

The API server should accept messages, verify signatures, and carry out each transaction if the MAC is valid. It's also publicly exposed - the attacker can submit messages freely assuming he can forge the right MAC.

The web client should allow the attacker to generate valid messages for accounts he controls. (Feel free to sanitize params if you're feeling anal-retentive.) Assume the attacker is in a position to capture and inspect messages from the client to the API server.

One thing we haven't discussed is the IV. Assume the client generates a per-message IV and sends it along with the MAC. That's how CBC works, right?

Wrong.

For messages signed under CBC-MAC, an attacker-controlled IV is a liability. Why? Because it yields full control over the first block of the message.

Use this fact to generate a message transferring 1M spacebucks from a target victim's account into your account.

I'll wait. Just let me know when you're done.

... waiting

... waiting

... waiting

All done? Great - I knew you could do it!

Now let's tune up that protocol a little bit.

As we now know, you're supposed to use a fixed IV with CBC-MAC, so let's do that. We'll set ours at 0 for simplicity. This means the IV comes out of the protocol:

message || MAC
Pretty simple, but we'll also adjust the message. For the purposes of efficiency, the bank wants to be able to process multiple transactions in a single request. So the message now looks like this:

from=#{from_id}&tx_list=#{transactions}
With the transaction list formatted like:

to:amount(;to:amount)*
There's still a weakness here: the MAC is vulnerable to length extension attacks. How?

Well, the output of CBC-MAC is a valid IV for a new message.

"But we don't control the IV anymore!"

With sufficient mastery of CBC, we can fake it.

Your mission: capture a valid message from your target user. Use length extension to add a transaction paying the attacker's account 1M spacebucks.

Hint!
This would be a lot easier if you had full control over the first block of your message, huh? Maybe you can simulate that.

In [13]:
from Crypto.Cipher import AES
from Crypto.Util.strxor import strxor

import re

def padPKCS7(x, k):
    ch = k - (len(x) % k)
    return x + chr(ch) * ch

def CBC_MAC(key, iv, p):
    cipher = AES.new(key, AES.MODE_CBC, iv)
    c = cipher.encrypt(padPKCS7(p, 16))
    return c[-16:]

key = randbytes(16)

def backend_process_message(m):
    global key
    message = m[:-32]
    iv = m[-32:-16]
    mac = m[-16:]
    if CBC_MAC(key, iv, message) != mac:
        print('S: Discarding invalid message')
        return
    sender = ''
    recipient = ''
    amount = 0
    for pair in message.split(b'&'):
        k, v = pair.split(b'=')
        if k == b'from':
            sender = v
        elif k == b'to':
            recipient = v
        elif k == b'amount':
            m = re.match(b'[0-9]+', v)
            if m:
                amount = int(m.group(0))
    print('S: Transferring', amount, 'from', sender, 'to', recipient)

last_sent_message = b''

def frontend_send_message(sender, recipient, amount):
    global key
    global last_sent_message
    if not re.match(b'^[A-Za-z]+$', sender):
        raise Exception(b'Invalid sender ' + sender)
    if not re.match(b'^[A-Za-z]+$', recipient):
        raise Exception(b'Invalid recipient ' + recipient)
    amount = int(amount)
    message = b'from=' + sender + b'&to=' + recipient + b'&amount=' + str(amount).encode('ascii')
    iv = randbytes(16)
    last_sent_message = message + iv + CBC_MAC(key, iv, message)
    print('C:', last_sent_message)
    backend_process_message(last_sent_message)

def attacker_peek_last_sent_message():
    global last_sent_message
    return last_sent_message

def attacker_inject_message(m):
    print('A: Injecting', m)
    backend_process_message(m)

def attacker_send_forged_message(sender, recipient, amount):
    # Assume attacker can create this account.
    fake_sender = (b'M' * min(len(sender), 11)) + sender[11:]
    # Assume this message fails or otherwise has no effect.
    frontend_send_message(fake_sender, recipient, amount)
    m = attacker_peek_last_sent_message()
    message, iv, mac = m[:-32], m[-32:-16], m[-16:]
    forged_message = b'from=' + sender + message[len(sender)+5:]
    forged_iv = strxor(iv, strxor(message[:16], forged_message[:16]))
    attacker_inject_message(forged_message + forged_iv + mac)

key2 = randbytes(16)
iv2 = b'\x00' * 16

def backend2_process_message(m):
    global key2
    global iv2
    message = m[:-16]
    mac = m[-16:]
    if CBC_MAC(key2, iv2, message) != mac:
        print('S2: Discarding invalid message')
        return
    sender = b''
    tx_list = b''
    for pair in message.split(b'&'):
        k, v = pair.split(b'=')
        if k == b'from':
            sender = v
        elif k == b'tx_list':
            tx_list = v
    tx_pairs = [p.split(b':', 1) for p in tx_list.split(b';')]
    for p in tx_pairs:
        amount = 0
        m = re.match(b'[0-9]+', p[1])
        if m:
            amount = int(m.group(0))
        print('S2: Transferring', amount, 'from', sender, 'to', p[0])

last_sent_message2 = b''

def attacker2_peek_last_sent_message():
    global last_sent_message2
    return last_sent_message2

def attacker2_process_message(m):
    print('A2', m)
    if re.match(b'^from=Tom&', m):
        message = m[:-16]
        mac1 = m[-16:]
        # Assume attacker can create this account, and that this
        # message fails or otherwise has no effect.
        frontend2_send_message(b'M', [[b'M', b'0'], [b'Mallory', b'1000000']])
        m2 = attacker2_peek_last_sent_message()
        m3 = padPKCS7(message, 16) + strxor(mac1, m2[:16]) + m2[16:]
        print('A2', m3)
        backend2_process_message(m3)
    else:
        backend2_process_message(m)

def frontend2_send_message(sender, tuples):
    global key2
    global iv2
    global last_sent_message2
    if not re.match(b'^[A-Za-z]+$', sender):
        raise Exception(b'Invalid sender ' + sender)
    for t in tuples:
        if not re.match(b'^[A-Za-z]+$', t[0]):
            raise Exception(b'Invalid recipient ' + t[0])
    recipients = b';'.join([t[0] + b':' + str(int(t[1])).encode('ascii') for t in tuples])
    message = b'from=' + sender + b'&tx_list=' + recipients
    last_sent_message2 = message + CBC_MAC(key2, iv2, message)
    attacker2_process_message(last_sent_message2)


attacker_send_forged_message(b'Tom', b'Mallory', b'1000000')

frontend2_send_message(b'Alice', [[b'Bob', b'5'], [b'Charlie', b'10']])
frontend2_send_message(b'Tom', [[b'Jerry', b'15'], [b'Susan', b'20']])

('C:', 'from=MMM&to=Mallory&amount=1000000\r\x11\xe1\x97\xa3?\x12\xd9S\xf1\x9d`\x1f8mY\xdf\xc7\x8c\xa26\xb1W\xba\xe2\xfe\x9e\x84m\x1f\xaf8')
('S: Transferring', 1000000, 'from', 'MMM', 'to', 'Mallory')
('A: Injecting', 'from=Tom&to=Mallory&amount=1000000\r\x11\xe1\x97\xa3&0\xf9S\xf1\x9d`\x1f8mY\xdf\xc7\x8c\xa26\xb1W\xba\xe2\xfe\x9e\x84m\x1f\xaf8')
('S: Transferring', 1000000, 'from', 'Tom', 'to', 'Mallory')
('A2', 'from=Alice&tx_list=Bob:5;Charlie:10\xfc\x15\x12O?\xf9 (\xfe;\x89C\xfd]\xc3\xfa')
('S2: Transferring', 5, 'from', 'Alice', 'to', 'Bob')
('S2: Transferring', 10, 'from', 'Alice', 'to', 'Charlie')
('A2', "from=Tom&tx_list=Jerry:15;Susan:20\x1a\xba@t.\xf1\xbd\xfd\xe4H\xc2\xa7\xb8\xfb\xe8'")
('A2', 'from=M&tx_list=M:0;Mallory:1000000j^f\xfd\x94\x80@\x07\xe1H.f|E\n\xd6')
('S2: Transferring', 0, 'from', 'M', 'to', 'M')
('S2: Transferring', 1000000, 'from', 'M', 'to', 'Mallory')
('A2', 'from=Tom&tx_list=Jerry:15;Susan:20\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e|\xc8/\

### Challenge 50 - Hashing with CBC-MAC

Sometimes people try to use CBC-MAC as a hash function.

This is a bad idea. Matt Green explains:

 To make a long story short: cryptographic hash functions are public functions (i.e., no secret key) that have the property of collision-resistance (it's hard to find two messages with the same hash). MACs are keyed functions that (typically) provide message unforgeability -- a very different property. Moreover, they guarantee this only when the key is secret.
Let's try a simple exercise.

Hash functions are often used for code verification. This snippet of JavaScript (with newline):

alert('MZA who was that?');
Hashes to 296b8d7cb78a243dda4d0a61d33bbdd1 under CBC-MAC with a key of "YELLOW SUBMARINE" and a 0 IV.

Forge a valid snippet of JavaScript that alerts "Ayo, the Wu is back!" and hashes to the same value. Ensure that it runs in a browser.

Extra Credit
Write JavaScript code that downloads your file, checks its CBC-MAC, and inserts it into the DOM iff it matches the expected hash.

In [ ]:
from Crypto.Util.strxor import strxor
import binascii
import itertools
import string

def insecureHash(s):
    return CBC_MAC(b'YELLOW SUBMARINE', b'\x00' * 16, s)

def prependAndCollide(s, prefix):
    prefixHash = insecureHash(prefix)
    paddedPrefix = padPKCS7(prefix, 16)
    return paddedPrefix + strxor(s[:16], prefixHash) + s[16:]

def prependAndCollidePrintable(s, prefix):
    # Ensure that the padding character is \t (== 0x09).
    numFreeBytes = (7 - len(prefix)) % 16
    if numFreeBytes == 0:
        numFreeBytes = 16
    for freeBytes in itertools.product(range(32, 127), repeat=numFreeBytes):
        paddedPrefix = prefix + "".join([chr(x) for x in freeBytes])
        collision = prependAndCollide(s, paddedPrefix)
        scrambledBytes = collision[-len(s):-len(s) + 16]
        if all(c >= 32 and c < 127 for c in scrambledBytes):
            return collision

s = b"alert('MZA who was that?');\n"
sHash = insecureHash(s)
print(binascii.hexlify(sHash))

prefix = b"alert('Ayo, the Wu is back!'); //"
collision = prependAndCollidePrintable(s, prefix)
print(collision, binascii.hexlify(insecureHash(collision)))

### Challenge 51 - Compression Ratio Side-Channel Attacks
Internet traffic is often compressed to save bandwidth. Until recently, this included HTTPS headers, and it still includes the contents of responses.

Why does that matter?

Well, if you're an attacker with:

Partial plaintext knowledge and
Partial plaintext control and
Access to a compression oracle
You've got a pretty good chance to recover any additional unknown plaintext.

What's a compression oracle? You give it some input and it tells you how well the full message compresses, i.e. the length of the resultant output.

This is somewhat similar to the timing attacks we did way back in set 4 in that we're taking advantage of incidental side channels rather than attacking the cryptographic mechanisms themselves.

Scenario: you are running a MITM attack with an eye towards stealing secure session cookies. You've injected malicious content allowing you to spawn arbitrary requests and observe them in flight. (The particulars aren't terribly important, just roll with it.)

So! Write this oracle:

oracle(P) -> length(encrypt(compress(format_request(P))))
Format the request like this:

POST / HTTP/1.1
Host: hapless.com
Cookie: sessionid=TmV2ZXIgcmV2ZWFsIHRoZSBXdS1UYW5nIFNlY3JldCE=
Content-Length: ((len(P)))
((P))
(Pretend you can't see that session id. You're the attacker.)

Compress using zlib or whatever.

Encryption... is actually kind of irrelevant for our purposes, but be a sport. Just use some stream cipher. Dealer's choice. Random key/IV on every call to the oracle.

And then just return the length in bytes.

Now, the idea here is to leak information using the compression library. A payload of "sessionid=T" should compress just a little bit better than, say, "sessionid=S".

There is one complicating factor. The DEFLATE algorithm operates in terms of individual bits, but the final message length will be in bytes. Even if you do find a better compression, the difference may not cross a byte boundary. So that's a problem.

You may also get some incidental false positives.

But don't worry! I have full confidence in you.

Use the compression oracle to recover the session id.

I'll wait.

Got it? Great.

Now swap out your stream cipher for CBC and do it again.

In [ ]:
from Crypto.Cipher import AES
from Crypto.Util import Counter
import base64
import string
import zlib

sessionid = 'TmV2ZXIgcmV2ZWFsIHRoZSBXdS1UYW5nIFNlY3JldCE='

def format_request(P):
    return '''POST / HTTP/1.1
Host: hapless.com
Cookie: sessionid={0}
Content-Length: {1}
{2}'''.format(sessionid, len(P), P)

def oracle_ctr(P):
    request = format_request(P)
    compressed_request = zlib.compress(request.encode('ascii'))
    key = randbytes(16)
    ctr = Counter.new(128)
    cipher = AES.new(key, AES.MODE_CTR, counter=ctr)
    encrypted_request = cipher.encrypt(compressed_request)
    return len(encrypted_request)

def oracle_cbc(P):
    request = format_request(P)
    compressed_request = zlib.compress(request.encode('ascii'))
    key = randbytes(16)
    iv = randbytes(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    encrypted_request = cipher.encrypt(padPKCS7(compressed_request, 16))
    return len(encrypted_request)

alphabet = string.ascii_letters + string.digits + '+/='

paddingAlphabet = '!@#$%^&*()-`~[]{}'

def getPadding(oracle, s):
    l = oracle(s)
    padding = ''
    for i in range(len(paddingAlphabet)):
        padding += paddingAlphabet[i]
        il = oracle(padding + s)
        if il > l:
            return padding

def guessNextByte(oracle, knownStr):
    min_ch = ''
    min_ch_sz = 0
    padding = getPadding(oracle, ('sessionid=' + knownStr + '~') * 8)
    for i in range(len(alphabet)):
        ch = alphabet[i]
        s = 'sessionid=' + knownStr + ch
        sz = oracle(padding + s * 8)
        if min_ch == '' or sz < min_ch_sz:
            min_ch = ch
            min_ch_sz = sz
    return min_ch

def recover_sessionid(oracle):
    knownStr = ''
    for i in range(0, 44):
        knownStr += guessNextByte(oracle, knownStr)
    return knownStr

recovered_sessionid = recover_sessionid(oracle_ctr)
if recovered_sessionid != sessionid:
    raise Exception(recovered_sessionid + ' != ' + sessionid)
print(base64.b64decode(recovered_sessionid))

recovered_sessionid = recover_sessionid(oracle_cbc)
if recovered_sessionid != sessionid:
    raise Exception(recovered_sessionid + ' != ' + sessionid)
print(base64.b64decode(recovered_sessionid))

### Challenge 52 - Iterated Hash Function Multicollisions

In [ ]:
from Crypto.Cipher import Blowfish
import struct

def to_bytes(n, length, endianess='big'):
    h = '%x' % n
    s = ('0'*(len(h) % 2) + h).zfill(length*2).decode('hex')
    return s if endianess == 'big' else s[::-1]

def MerkleDamgard(f, processIV, blockLength, padMessage):
    def hashFn(m, iv, pad=True):
        H = processIV(iv)
        if pad:
            m = padMessage(m, len(m))
        elif len(m) % blockLength != 0:
            raise Exception('message of length {0} not a multiple of block length {1}'.format(len(m), blockLength))
        for block in (m[x:x+blockLength] for x in range(0, len(m), blockLength)):
            H = f(block, H)
        return H
    return hashFn

badHashHashLength = 2

def badHashF(messageBlock, state):
    cipher = Blowfish.new(state, Blowfish.MODE_ECB)
    newState = cipher.encrypt(messageBlock)[:badHashHashLength]
    return newState

def badHashProcessIV(iv):
    if len(iv) < badHashHashLength:
        return iv + (b'\x00' * (badHashHashLength - len(iv)))
    else:
        return iv[:badHashHashLength]

badHashBlockLength = 8

def badHashPadMessage(m, l):
    m += b'\x80'
    m += b'\x00' * ((-4 - (len(m) % badHashBlockLength)) % badHashBlockLength)
    m += struct.pack('>I', l)
    return m

badHash = MerkleDamgard(badHashF, badHashProcessIV, badHashBlockLength, badHashPadMessage)

def findPrefixCollisionFromIter(hashFn, iv, inputIter):
    hashToString = {}
    for s in inputIter:
        h = hashFn(s, iv, pad=False)
        if h in hashToString:
            return (h, s, hashToString[h])
        else:
            hashToString[h] = s
    return None, None, None

def findPrefixCollision(hashFn, iv, blockLength, hashLength):
    state, s1, s2 = findPrefixCollisionFromIter(hashFn, iv, (to_bytes(i, blockLength, endianess='little') for i in range(2**(hashLength*8))))
    if state is None:
        raise Error('unexpected')
    return state, s1, s2

def extendCollisions(hashFn, state, blockLength, hashLength, collisions):
    state, s1, s2 = findPrefixCollision(hashFn, state, blockLength, hashLength)
    return state, [x + s for x in collisions for s in [s1, s2]]

def generateCollisions(hashFn, iv, blockLength, hashLength, n):
    state, s1, s2 = findPrefixCollision(hashFn, iv, blockLength, hashLength)
    collisions = [s1, s2]
    for i in range(n-1):
        state, collisions = extendCollisions(hashFn, state, blockLength, hashLength, collisions)
    return state, collisions

lessBadHashHashLength = 3

def lessBadHashF(messageBlock, state):
    cipher = Blowfish.new(state, Blowfish.MODE_ECB)
    newState = cipher.encrypt(messageBlock)[:lessBadHashHashLength]
    return newState

lessBadHashProcessIV = badHashProcessIV
lessBadHashBlockLength = badHashBlockLength
lessBadHashPadMessage = badHashPadMessage

lessBadHash = MerkleDamgard(lessBadHashF, lessBadHashProcessIV, lessBadHashBlockLength, lessBadHashPadMessage)

# hashFn is assumed to be the cheap one.
def findCollision2(hashFn, iv, blockLength, hashLength, hashFn2, iv2, blockLength2, hashLength2):
    state, collisions = generateCollisions(hashFn, iv, blockLength, hashLength, hashLength2*4)
    while True:
        _, s1, s2 = findPrefixCollisionFromIter(hashFn2, iv2, collisions)
        if s1 is not None:
            return s1, s2
        state, collisions = extendCollisions(hashFn, state, blockLength, hashLength, collisions)


_, collisions = generateCollisions(badHash, b'', badHashBlockLength, badHashHashLength, 5)
for s in collisions:
    print(s, badHash(s, b''))

s1, s2 = findCollision2(badHash, b'', badHashBlockLength, badHashHashLength, lessBadHash, b'', lessBadHashBlockLength, lessBadHashHashLength)
print(s1, s2, badHash(s1, b''), badHash(s2, b''), lessBadHash(s1, b''), lessBadHash(s2, b''))

### Challenge 53 - Kelsey and Schneier's Expandable Messages
One of the basic yardsticks we use to judge a cryptographic hash function is its resistance to second preimage attacks. That means that if I give you x and y such that H(x) = y, you should have a tough time finding x' such that H(x') = H(x) = y.

How tough? Brute-force tough. For a 2^b hash function, we want second preimage attacks to cost 2^b operations.

This turns out not to be the case for very long messages.

Consider the problem we're trying to solve: we want to find a message that will collide with H(x) in the very last block. But there are a ton of intermediate blocks, each with its own intermediate hash state.

What if we could collide into one of those? We could then append all the following blocks from the original message to produce the original H(x). Almost.

We can't do this exactly because the padding will mess things up.

What we need are expandable messages.

In the last problem we used multicollisions to produce 2^n colliding messages for n*2^(b/2) effort. We can use the same principles to produce a set of messages of length (k, k + 2^k - 1) for a given k.

Here's how:

Starting from the hash function's initial state, find a collision between a single-block message and a message of 2^(k-1)+1 blocks. DO NOT hash the entire long message each time. Choose 2^(k-1) dummy blocks, hash those, then focus on the last block.
Take the output state from the first step. Use this as your new initial state and find another collision between a single-block message and a message of 2^(k-2)+1 blocks.
Repeat this process k total times. Your last collision should be between a single-block message and a message of 2^0+1 = 2 blocks.
Now you can make a message of any length in (k, k + 2^k - 1) blocks by choosing the appropriate message (short or long) from each pair.

Now we're ready to attack a long message M of 2^k blocks.

Generate an expandable message of length (k, k + 2^k - 1) using the strategy outlined above.
Hash M and generate a map of intermediate hash states to the block indices that they correspond to.
From your expandable message's final state, find a single-block "bridge" to intermediate state in your map. Note the index i it maps to.
Use your expandable message to generate a prefix of the right length such that len(prefix || bridge || M[i..]) = len(M).
The padding in the final block should now be correct, and your forgery should hash to the same value as M.

In [ ]:

def findStatePrefixCollision(hashFn, iv1, iv2, blockLength):
    hashToIV2Block = {}
    for s in (to_bytes(i, blockLength, endianess='little') for i in range(2**(blockLength*8))):
        h = hashFn(s, iv1, pad=False)
        if h in hashToIV2Block:
            return (h, s, hashToIV2Block[h])

        h = hashFn(s, iv2, pad=False)
        hashToIV2Block[h] = s
    raise Exception('unexpected')

def findNBlockPrefixCollision(hashFn, iv, blockLength, n):
    prefix = b'\x00' * (blockLength * (n-1))
    prefixHash = hashFn(prefix, iv, pad=False)
    h, s1, lastBlock = findStatePrefixCollision(hashFn, iv, prefixHash, blockLength)
    return h, s1, prefix + lastBlock

def makeExpandablePrefix(hashFn, iv, blockLength, k):
    blocks = []
    state = iv
    for i in range(k):
        state, s, nBlock = findNBlockPrefixCollision(hashFn, state, blockLength, 2**(k-1-i)+1)
        blocks += [[s, nBlock]]
    return state, blocks

def makeExpandedPrefix(blockSize, blocks, k, l):
    m = b''
    for i in range(len(blocks)):
        block = blocks[i]
        if len(m) // blockSize + len(block[1]) // blockSize + (len(blocks) - i - 1) <= l:
            nextSegment = block[1]
        else:
            nextSegment = block[0]
        m += nextSegment
    if len(m) // blockSize != l:
        raise Exception('unexpected')
    return m

def getIntermediateStates(m, hashFn, iv, blockLength):
    state = iv
    for block in (m[blockLength*i:blockLength*(i+1)] for i in range(len(m) // blockLength)):
        state = hashFn(block, state, pad=False)
        yield state

def findCollisionInSet(hashFn, iv, blockLength, hashSet):
    for s in (to_bytes(i, blockLength, endianess='little') for i in range(2**(blockLength*8))):
        h = hashFn(s, iv, pad=False)
        if h in hashSet:
            return s, h

    raise Exception('unexpected')

def findIntermediateStateCollision(hashFn, iv, blockLength, hashLength, intermediateStateIter, minBlockCount):
    statesToIndices = {}
    i = 0
    for state in intermediateStateIter:
        if i >= minBlockCount:
            statesToIndices[state] = i
        i += 1

    if not statesToIndices:
        raise Exception('unexpected')

    s, h = findCollisionInSet(hashFn, iv, blockLength, statesToIndices)
    return s, statesToIndices[h]

def findSecondPreimage(m, hashFn, iv, blockLength, hashLength):
    blockCount = (len(m) + blockLength - 1) // blockLength
    k = blockCount.bit_length()
    prefixState, blocks = makeExpandablePrefix(hashFn, iv, blockLength, k)

    intermediateStateIter = getIntermediateStates(m, hashFn, iv, blockLength)
    bridge, collisionBlockCount = findIntermediateStateCollision(hashFn, prefixState, blockLength, hashLength, intermediateStateIter, k)

    m2 = makeExpandedPrefix(blockLength, blocks, k, collisionBlockCount) + bridge
    m2 += m[len(m2):]
    return m2

m = randbytes(100)
h = badHash(m, b'')
m2 = findSecondPreimage(m, badHash, b'', badHashBlockLength, badHashHashLength)
h2 = badHash(m2, b'')
print(m, m2, h, h2)
if len(m2) != len(m):
    raise Exception('{0} != {1}'.format(len(m2), len(m)))
if h2 != h:
    raise Exception(h2 + b' != ' + h)

### Challenge 54 - Kelsey and Kohno's Nostradamus Attack
Hash functions are sometimes used as proof of a secret prediction.

For example, suppose you wanted to predict the score of every Major League Baseball game in a season. (2,430 in all.) You might be concerned that publishing your predictions would affect the outcomes.

So instead you write down all the scores, hash the document, and publish the hash. Once the season is over, you publish the document. Everyone can then hash the document to verify your soothsaying prowess.

But what if you can't accurately predict the scores of 2.4k baseball games? Have no fear - forging a prediction under this scheme reduces to another second preimage attack.

We could apply the long message attack from the previous problem, but it would look pretty shady. Would you trust someone whose predicted message turned out to be 2^50 bytes long?

It turns out we can run a successful attack with a much shorter suffix. Check the method:

Generate a large number of initial hash states. Say, 2^k.
Pair them up and generate single-block collisions. Now you have 2^k hash states that collide into 2^(k-1) states.
Repeat the process. Pair up the 2^(k-1) states and generate collisions. Now you have 2^(k-2) states.
Keep doing this until you have one state. This is your prediction.
Well, sort of. You need to commit to some length to encode in the padding. Make sure it's long enough to accommodate your actual message, this suffix, and a little bit of glue to join them up. Hash this padding block using the state from step 4 - THIS is your prediction.
What did you just build? It's basically a funnel mapping many initial states into a common final state. What's critical is we now have a big field of 2^k states we can try to collide into, but the actual suffix will only be k+1 blocks long.

The rest is trivial:

Wait for the end of the baseball season. (This may take some time.)
Write down the game results. Or, you know, anything else. I'm not too particular.
Generate enough glue blocks to get your message length right. The last block should collide into one of the leaves in your tree.
Follow the path from the leaf all the way up to the root node and build your suffix using the message blocks along the way.
The difficulty here will be around 2^(b-k). By increasing or decreasing k in the tree generation phase, you can tune the difficulty of this step. It probably makes sense to do more work up-front, since people will be waiting on you to supply your message once the event passes. Happy prognosticating!

In [ ]:
def constructCollisionTree(hashFn, blockLength, hashLength, k):
    leaves = [(to_bytes(i, hashLength, endianess='little'),)
              for i in range(2**k)]

    initialStateMap = {leaves[i][0]:i for i in range(2**k)}

    collisionTree = [leaves]

    for i in range(1, k+1):
        prev_level = collisionTree[i-1]
        curr_level = [
            findStatePrefixCollision(hashFn, prev_level[2*i][0], prev_level[2*i+1][0], blockLength) for i in range(2**(k-i))]
        collisionTree += [curr_level]

    return (initialStateMap, collisionTree)

def getSuffixFromCollisionTree(initialStateMap, collisionTree, iv):
    s = b''
    k = len(initialStateMap).bit_length() - 1

    i = initialStateMap[iv]
    for j in range(1, k+1):
        n = collisionTree[j][i//2]
        s += n[1 + (i%2)]
        i //= 2
    return s

def generatePrediction(hashFn, padFn, initialStateMap, collisionTree, messageLength, blockLength):
    k = len(initialStateMap).bit_length() - 1
    totalBlockLength = (messageLength + (blockLength - 1)) // blockLength
    # One glue block, plus k for the suffix.
    totalBlockLength += k + 1
    totalLength = totalBlockLength * blockLength
    return hashFn(padFn(b'', totalLength), collisionTree[-1][0][0], pad=False)

def forgePrediction(hashFn, iv, padFn, initialStateMap, collisionTree, messageLength, blockLength, m):
    if len(m) > messageLength:
        raise Exception('message too big')
    paddedMessageLength = messageLength + (-messageLength % blockLength)
    m += b'\x00' * (paddedMessageLength - len(m))
    h = hashFn(m, iv, pad=False)

    glue, h = findCollisionInSet(hashFn, h, blockLength, initialStateMap)
    m += glue
    suffix = getSuffixFromCollisionTree(initialStateMap, collisionTree, h)
    m += suffix

    return m

k = 5
messageLength = 100
(initialStateMap, collisionTree) = constructCollisionTree(badHash, badHashBlockLength, badHashHashLength, k)
prediction = generatePrediction(badHash, badHashPadMessage, initialStateMap, collisionTree, messageLength, badHashBlockLength)

print('Prediction', prediction)

forgedPrediction = b'Dodgers win the World Series!!'
forgedPredictionMessage = forgePrediction(badHash, b'', badHashPadMessage, initialStateMap, collisionTree, messageLength, badHashBlockLength, forgedPrediction)
h = badHash(forgedPredictionMessage, b'')
print('Forged prediction', forgedPredictionMessage, h)

### Challenge 55 - MD4 Collisions
MD4 is a 128-bit cryptographic hash function, meaning it should take a work factor of roughly 2^64 to find collisions.

It turns out we can do much better.

The paper "Cryptanalysis of the Hash Functions MD4 and RIPEMD" by Wang et al details a cryptanalytic attack that lets us find collisions in 2^8 or less.

Given a message block M, Wang outlines a strategy for finding a sister message block M', differing only in a few bits, that will collide with it. Just so long as a short set of conditions holds true for M.

What sort of conditions? Simple bitwise equalities within the intermediate hash function state, e.g. a[1][6] = b[0][6]. This should be read as: "the sixth bit (zero-indexed) of a[1] (i.e. the first update to 'a') should equal the sixth bit of b[0] (i.e. the initial value of 'b')".

It turns out that a lot of these conditions are trivial to enforce. To see why, take a look at the first (of three) rounds in the MD4 compression function. In this round, we iterate over each word in the message block sequentially and mix it into the state. So we can make sure all our first-round conditions hold by doing this:
```
# calculate the new value for a[1] in the normal fashion
a[1] = (a[0] + f(b[0], c[0], d[0]) + m[0]).lrot(3)

# correct the erroneous bit
a[1] ^= ((a[1][6] ^ b[0][6]) << 6)

# use algebra to correct the first message block
m[0] = a[1].rrot(3) - a[0] - f(b[0], c[0], d[0])
```

Simply ensuring all the first round conditions puts us well within the range to generate collisions, but we can do better by correcting some additional conditions in the second round. This is a bit trickier, as we need to take care not to stomp on any of the first-round conditions.

Once you've adequately massaged M, you can simply generate M' by flipping a few bits and test for a collision. A collision is not guaranteed as we didn't ensure every condition. But hopefully we got enough that we can find a suitable (M, M') pair without too much effort.

Implement Wang's attack.
### Challenge 56 - RC4 Single-Byte Biases

RC4 is popular stream cipher notable for its usage in protocols like TLS, WPA, RDP, &c.

It's also susceptible to significant single-byte biases, especially early in the keystream. What does this mean?

Simply: for a given position in the keystream, certain bytes are more (or less) likely to pop up than others. Given enough encryptions of a given plaintext, an attacker can use these biases to recover the entire plaintext.

Now, search online for "On the Security of RC4 in TLS and WPA". This site is your one-stop shop for RC4 information.

Click through to "RC4 biases" on the right.

These are graphs of each single-byte bias (one per page). Notice in particular the monster spikes on z16, z32, z48, etc. (Note: these are one-indexed, so z16 = keystream[15].)

How useful are these biases?

Click through to the research paper and scroll down to the simulation results. (Incidentally, the whole paper is a good read if you have some spare time.) We start out with clear spikes at 2^26 iterations, but our chances for recovering each of the first 256 bytes approaches 1 as we get up towards 2^32.

There are two ways to take advantage of these biases. The first method is really simple:

Gain exhaustive knowledge of the keystream biases.
Encrypt the unknown plaintext 2^30+ times under different keys.
Compare the ciphertext biases against the keystream biases.
Doing this requires deep knowledge of the biases for each byte of the keystream. But it turns out we can do pretty well with just a few useful biases - if we have some control over the plaintext.

How? By using knowledge of a single bias as a peephole into the plaintext.

Decode this secret:

QkUgU1VSRSBUTyBEUklOSyBZT1VSIE9WQUxUSU5F
And call it a cookie. No peeking!

Now use it to build this encryption oracle:

RC4(your-request || cookie, random-key)
Use a fresh 128-bit key on every invocation.

Picture this scenario: you want to steal a user's secure cookie. You can spawn arbitrary requests (from a malicious plugin or somesuch) and monitor network traffic. (Ok, this is unrealistic - the cookie wouldn't be right at the beginning of the request like that - this is just an example!)

You can control the position of the cookie by requesting "/", "/A", "/AA", and so on.

Build bias maps for a couple chosen indices (z16 and z32 are good) and decrypt the cookie.

MD4 is a 128-bit cryptographic hash function, meaning it should take a work factor of roughly 2^64 to find collisions.

It turns out we can do much better.

The paper "Cryptanalysis of the Hash Functions MD4 and RIPEMD" by Wang et al details a cryptanalytic attack that lets us find collisions in 2^8 or less.

Given a message block M, Wang outlines a strategy for finding a sister message block M', differing only in a few bits, that will collide with it. Just so long as a short set of conditions holds true for M.

What sort of conditions? Simple bitwise equalities within the intermediate hash function state, e.g. a[1][6] = b[0][6]. This should be read as: "the sixth bit (zero-indexed) of a[1] (i.e. the first update to 'a') should equal the sixth bit of b[0] (i.e. the initial value of 'b')".

It turns out that a lot of these conditions are trivial to enforce. To see why, take a look at the first (of three) rounds in the MD4 compression function. In this round, we iterate over each word in the message block sequentially and mix it into the state. So we can make sure all our first-round conditions hold by doing this:
```
# calculate the new value for a[1] in the normal fashion
a[1] = (a[0] + f(b[0], c[0], d[0]) + m[0]).lrot(3)

# correct the erroneous bit
a[1] ^= ((a[1][6] ^ b[0][6]) << 6)

# use algebra to correct the first message block
m[0] = a[1].rrot(3) - a[0] - f(b[0], c[0], d[0])
```
Simply ensuring all the first round conditions puts us well within the range to generate collisions, but we can do better by correcting some additional conditions in the second round. This is a bit trickier, as we need to take care not to stomp on any of the first-round conditions.

Once you've adequately massaged M, you can simply generate M' by flipping a few bits and test for a collision. A collision is not guaranteed as we didn't ensure every condition. But hopefully we got enough that we can find a suitable (M, M') pair without too much effort.

Implement Wang's attack.

# Set 8